# Incluindo validação


In [ ]:
import torchvision.transforms as transforms
import torchvision.transforms.functional as transF
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from PIL import Image
from imgaug import augmenters as iaa
import numpy as np

In [ ]:
my_aug = iaa.Sequential([
            iaa.Sometimes(0.25, iaa.Affine(scale={"x": (1.0, 2.0), "y": (1.0, 2.0)})),
            iaa.Resize((224, 224)),
            iaa.Fliplr(0.5),
            iaa.Flipud(0.2),
            iaa.Sometimes(0.25, iaa.Affine(rotate=(-120, 120), mode='symmetric')),
            iaa.Sometimes(0.25, iaa.GaussianBlur(sigma=(0, 3.0))),

            # noise
            iaa.Sometimes(0.1,
                          iaa.OneOf([
                              iaa.Dropout(p=(0, 0.05)),
                              iaa.CoarseDropout(0.02, size_percent=0.25)
                          ])),

            iaa.Sometimes(0.25,
                          iaa.OneOf([
                              iaa.Add((-15, 15), per_channel=0.5), # brightness
                              iaa.AddToHueAndSaturation(value=(-10, 10), per_channel=True)
                          ])),

        ])

In [ ]:
my_trans = transforms.Compose([
                np.array,
                my_aug.augment_image,
                np.copy,
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])

In [ ]:
my_trans_eval = transforms.Compose([             
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])

## Vamos usar o transform com o `my_dataset`
- Vamos usar os resultados do notebook anterior para carregar os dados da base de felinos novamente

In [ ]:
class MyDataset (torch.utils.data.Dataset):
    """
    Esse dataset recebe uma lista de path de imagens, uma lista de labels correspondentes e (opcional) transformações pata aplicar nas imagens
    quando elas forem carregadas
    """

    def __init__(self, imgs_path, labels, my_transform=None):
        """
        imgs_path: list ou tuple
            Uma lista ou tupla com os paths para todas as imagens
        labels: lista ou tuple
            Uma lista ou tupla com o label de todas as imagens. Obviamente, precisa dar match com o os paths
        my_transform: None ou torchvision.transforms
            Uma sequência de transformadores para aplicar nos dados. Se for None, ele apenas transforma em tensor
        """

        super().__init__()
        self.imgs_path = imgs_path
        self.labels = labels
        
        # se my_transform for None, precisamos garantir que a imagem PIL seja transformada em Tensor para nao 
        # obtermos um erro quando usarmos o dataloader (ver aulas passadas)        
        if my_transform is not None:
            self.transform = my_transform
        else:
            self.transform = transforms.ToTensor()


    def __len__(self):     
        """
        Sobrecarga do método len para obtermos o tamanho do dataset. Não é obrigatório implementar
        """
        return len(self.imgs_path)


    def __getitem__(self, item):        
        """
        Esse método obtém uma imagem e um label cada vez que iteramos no Dataset. Ele também aplica a transformação
        na imagem. É obrigatório sua implementação
        
        item: int 
            Um indice no intervalo [0, ..., len(img_paths)-1]
        
        return: tuple 
             Uma tupla com a imagem, label e ID da imagem correspondentes ao item
        """

        # Aqui usamos PIL para carregar as imagens
        image = Image.open(self.imgs_path[item]).convert("RGB")

        # Aplicando as transformações
        image = self.transform(image)

        # Obtendo o ID da imagem
        img_id = self.imgs_path[item].split('/')[-1].split('.')[0]

        if self.labels is None:
            labels = []
        else:
            labels = self.labels[item]

        return image, labels, img_id

#### Preparando dados para criarmos um dataset

In [ ]:
from glob import glob
import os

In [ ]:
train_data_path = "/home/patcha/datasets/felinos/train"
test_data_path = "/home/patcha/datasets/felinos/test"

In [ ]:
labels_name = glob(os.path.join(train_data_path, "*"))
labels_name = [l.split(os.path.sep)[-1] for l in labels_name]
labels_name

In [ ]:
def get_paths_and_labels(path, lab_names):
    imgs_path, labels = list(), list()
    lab_cnt = 0
    for lab in lab_names:    
        paths_aux = glob(os.path.join(path, lab, "*.jpg"))
        
        # Atualizando os labels e imgs_paths
        labels += [lab_cnt] * len(paths_aux)
        imgs_path += paths_aux
        
        lab_cnt += 1
        
    return imgs_path, labels

- Obtendo paths e labels para cada partição

In [ ]:
train_imgs_paths, train_labels = get_paths_and_labels(train_data_path, labels_name)
temp_imgs_paths, temp_labels = get_paths_and_labels(test_data_path, labels_name)

- **Novo**: criando o conjunto de validação

In [ ]:
from sklearn.model_selection import train_test_split
test_imgs_paths, val_imgs_paths, test_labels, val_labels = train_test_split(temp_imgs_paths, temp_labels, test_size=0.5, random_state=10)
len(test_imgs_paths), len(val_imgs_paths)

#### Instanciando os datasets

In [ ]:
train_dataset = MyDataset(train_imgs_paths, train_labels, my_transform=my_trans)
test_dataset = MyDataset(test_imgs_paths, test_labels, my_transform=my_trans_eval)
val_dataset = MyDataset(val_imgs_paths, val_labels, my_transform=my_trans_eval)

### Criando um Dataloader
- Agora, podemos criar um [dataloader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) como fizemos nas últimas duas aulas

In [ ]:
batch_size = 30
train_dataloader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                               batch_size=batch_size, 
                                               shuffle=True)

test_dataloader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                               batch_size=batch_size, 
                                               shuffle=True)

val_dataloader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                               batch_size=batch_size, 
                                               shuffle=True)

## (Re)criando uma CNN
- Vamos criar uma CNN para classificar os felinos
- Para facilitar o projeto, vamos criar uma função que calcula o tamanho da saída

In [ ]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=4):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=7, stride=2), # 109
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)) # 54
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 16, kernel_size=7, stride=2), # 24
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=4)) # 6
        self.fc = nn.Linear(576, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)        
        out = self.layer2(out)
        # Fazendo a operação de flatten        
        out = out.reshape(out.size(0), -1)        
        out = self.fc(out)
        return out

- Agora podemos instanciar o nosso modelo:

In [ ]:
model = ConvNet()
model

- Agora podemos determinar nossa função de custo e otimizador
- Para esse notebook vamos aproveitar o que já fizemos no anterior

In [ ]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)  

- Agora vamos fazer nosso loop de treinamento
- Agora, vamos mandar nosso modelo para GPU se ela estiver disponível
- **Novo**: vamos criar nosso pipeline de checkpoints

In [ ]:
def eval_loss(data_loader):    
    with torch.no_grad():
        loss, cnt = 0, 0
        for images, labels, img_id in data_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss += loss_func(outputs, labels).item()
            cnt += 1
    return loss/cnt  

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [ ]:
num_epochs = 30
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Movendo o modelo para o device alvo
model.to(device)

best_loss = np.inf

for epoch in range(num_epochs):
    
    loss_epoch, cnt = 0, 0
    for k, (batch_images, batch_labels, id_img) in enumerate(train_dataloader):  
        
        # Aplicando um flatten na imagem e movendo ela para o device alvo
        batch_images = batch_images.to(device)
        batch_labels = batch_labels.to(device)
        
        # Fazendo a forward pass
        # observe que o modelo é agnóstico ao batch size
        outputs = model(batch_images)
        loss = loss_func(outputs, batch_labels)
        loss_epoch += loss.item()
        
        # Fazendo a otimização
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()    
        cnt += 1
        
        
    loss_epoch = loss_epoch / cnt
    loss_val_epoch = eval_loss(val_dataloader)
    
    
    writer.add_scalar("LossTrain", loss_epoch, epoch)
    writer.add_scalar("LossVal", loss_val_epoch, epoch)
    
    temp = {
        "Train": loss_epoch,
        "Val": loss_val_epoch
    }
    writer.add_scalars("Loss", temp, epoch)
    
        
    # Salvando o checkpoint da última época
    checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss_func,
            'loss_val': loss_epoch
    }        
    torch.save(checkpoint, "last_checkpoint.pth")

    # Salvando a mellhor execução    
    if loss < best_loss:        
        best_loss = loss
        torch.save(checkpoint, "best_checkpoint.pth")
        
    
    print (f"- Epoch [{epoch+1}/{num_epochs}] | Loss: {loss_epoch:.4f} | Loss Val: {loss_val_epoch:.4f}")                  

### Fazendo inferência no conjunto de teste
- A inferência é basicamente igual a do notebook anterior

In [ ]:
eval_loss(test_dataloader)

In [ ]:
with torch.no_grad():
    correct, total = 0, 0
    for images, labels, img_id in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Accuracy: {100 * correct / total}%") 